In [1]:
from model import Sol, SolCollection, read_stations
import model as m
import numpy as np
import pandas as pd
import utilities as util
from utilities import get_lims_pts, normalize_list_in_place, plot_list_points, blockPrint, enablePrint
import random
from copy import deepcopy
random.seed(12334789)
%matplotlib inline
import matplotlib.pyplot as plt

## Avances
- Túneo de parámetros
- Modelos Exactos C++

## Contenidos
1. [Tuneo parámetros](#tuneo)
2. [Optimización modelos C++](#c++)
3. [Conclusiones](#conclusiones)


### 1. Tuneo de parametros <a name="tuneo"></a>
1. Correr modelos con todas las diferentes parametrizaciones
2. Sacar métricas de cada uno de los modelos
3. Ponderar las métricas entre todas las instancias para cada modelo.
4. Escoger las mejores parametrizaciones que lleve a las mejores métricas.

#### EJEMPLO DE QUE SE ESTA HACIENDO PARA COMPARAR LAS METRICAS

In [2]:
# Read file and stations
fp = '~/Dropbox/PI/PI2/data/n30q10A.dat'
file_stations = util.rdead_file(fp)
stations = read_statiCons(file_stations) # list of stations
Sol.set_stations(stations)

In [3]:
soluciones1 = SolCollection(n_pob=50, ratio_sons=.75, ratio_mutation=.2, num_random_sols=4)
res1 = soluciones1.train_time(1)


In [4]:
soluciones2 = SolCollection(n_pob=50, ratio_sons=.75, ratio_mutation=.2, num_random_sols=4)
res2 = soluciones2.train_time(5)

In [5]:
util.get_metrics([res1, res2])

,set_cov,gen_dist,spacing,eucl_sum
0,0.310345,0.005610,0.027808,1.613442
1,0.629630,0.008048,0.025937,1.658225


## main_tunning.py

Para cada instancia se corren 81 diferentes combinaciones de parámetros.  

Las combinaciones de pamétros de definen en el archivo **main_tunning.py** y son las siguientes.

<pre><code class="python"># Combinaciones de parametros

possible_values = {
    'n_pob': [20, 40, 80],
    'ratios_sons': [.5 ,.75, 1],
    'ratio_mutation': [.05, .2, .6],
    'num_random_sols': [4, 10, 15]
}
</code></pre>

*INSTANCIAS* (56) : 20A, 20B, 20C ... 300H.    

Por lo que en definitiva se corren *81x56* veces el genético. 10 segundos cada uno. Los resultados de cada combinación en cada instancia se guardan en **all\_results.xlsx** y los consolidados (promedio) entre todas las instancias se guarda en el archivo **mean\_instances.py** donde se normaliza y se ordena, para ver cual son las combinaciones de parametros que sacarón mejor valor en la ponderación.

Todo lo anteriormente descrito se hace en **main\_tunning.py**



## 2. Optimización de modelos en C++ <a name="c++"></a>

La idea en este punto es realmente empezar a comparar soluciones, es decir, que tan rápido es el genético. Y que tan competitivo es con los modelos previamente existentes. Por lo que creamos diferentes modelos que serán corridos en todas las instancias y compararemos la solución de ese modelo y el tiempo con las del genético con sus mejores combinaciones de parámetros..

Se propone un modelo general (primera imagen) y a partir de ahí se añaden algunas restricciones.

<table style="width:100%">
  <tr>
    <th><img src="../../exact_models_cpp/images/1.jpeg" alt="Primer modelo" width="700"/></th>
    <th><img src="../../exact_models_cpp/images/2.jpeg" alt="Segundo modelo" width="700"/></th> 
    <th><img src="../../exact_models_cpp/images/3.jpeg" alt="Primer modelo" width="700"/></th>
  </tr>

</table>

Para esto partimos de la implementación de Juan David, y todo esto está en la ruta   https://github.com/NumberPiOso/ShortGenetic/tree/master/code/exact_models_cpp/

En especifico, el código de Juandavid es:
[MILP_1PDTSP](../../exact_models_cpp/MILP_1PDTSP.cpp)

Y es importante tener en cuenta las diferentes notaciones que se tienen, 


<table style="width:50%">
  <tr>
    <th>Notación Juan David</th>
    <th>Notación Juan Carlos</th>
    <th>Explicación</th> 
  </tr>
    
  <tr>
    <th>$X_{ij}$</th>
    <th>$L_{ij}$</th>
    <th>Carga que lleva el vehículo cuando va del nodo $i$ al nodo $j$.</th>
  </tr>
    
  <tr>
    <th>$Y_{ij}$</th>
    <th>$X_{ij}$</th>
    <th>1 si se va del nodo $i$ al nodo $j$.</th>
  </tr>
    
  <tr>
    <th>$Z_{ij}$</th>
    <th>$Y_{ij}$</th>
    <th>indica el orden en el que el arco $ij$ será visitado en al solución</th>
  </tr>

</table>

Por lo que se logra hacer cierta similitud en las restricciones que se tienen, presentada en la siguiente imagén.

<img src="../../exact_models_cpp/images/notaciones.jpeg" alt="Diferentes modelos" width="200"/>


### Implementaciones
Se crearon los modelos en C++.

- [OTSP](../../exact_models_cpp/MILP_1PDTSP_wRestrictions.cpp) : (1-14)
- [OPDTSP](exact_models_cpp/MILP_1PDTSP_wRestrictions_v2.cpp) : 1-7, 15, 10-14
- [OSPDTSP](exact_models_cpp/MILP_1PDTSP_wRestrictions_v3.cpp) : 1, 16, 17, 4-7, 18, 10-14, 19
- [P constrains(sin revisar)](exact_models_cpp/MILP_1PDTSP_wRestrictions_v4.cpp) : 1, 16-17, 4-7, 20, 10-14, 2 nuevas p constrains

Se alcanzaron a crear los modelos .mps, con ayuda de Juan David, (porque yo no tengo la licencia de Gurobi entonces no podía ni crear el .mps) y se estaba corriendo el **OTSP** cuando el computador de Grimmat se apagó, les pusimos limite de tiempo a cada instancia de 1 hora.



## CONCLUSIONES <a name="conclusiones"></a>

1. Se ha hecho un tuneo de parametros con los parametros más simples necesarios que se le puede meter a un genético, se planea no avanzar más en la construcción del genético hasta que se tenga una versión de principio a fin, funcional, fácil de entender y de explicar, que se pueda comparar con algo. (Ya después le metemos los métodos de busqueda local, que ya fueron implementados en PI2).

2. Necesitamos solucionar el problema de la licencia para poder de alguna manera poder compilar y correr los modelos exactos, estos modelos son totalmente necesarios para poder comparar el genético con algo.